<a href="https://colab.research.google.com/github/aaronjyang/transformers-testing/blob/main/LLaVa-loftus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load model and necessary imports

In [1]:
# Pip installs
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install -q av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.7 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/tran

In [2]:
# Imports
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor
import torch
from PIL import Image
import numpy as np
import av
import numpy as np
from huggingface_hub import hf_hub_download
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

In [3]:
# Load Model

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

processor = LlavaNextVideoProcessor.from_pretrained("llava-hf/LLaVA-NeXT-Video-7B-hf")
model = LlavaNextVideoForConditionalGeneration.from_pretrained(
    "llava-hf/LLaVA-NeXT-Video-7B-hf",
    quantization_config=quantization_config,
    device_map='auto'
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

# Load videos


In [4]:
# Helper function to read video file-paths from "videos" folder as numpy arrays
def read_video_pyav(path, indices):
    '''
    Decode the video with PyAV decoder.

    Args:
        path (str): Path to the video file.
        indices (List[int]): List of frame indices to decode.

    Returns:
        np.ndarray: np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    container = av.open(path)
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [5]:
# Given video names, load in all videos
videoIDs = ["720p10fps"]
videos = {}
indices = np.arange(0, 51)
for videoID in videoIDs:
  video = read_video_pyav("/content/videos/" + videoID + ".mp4", indices)
  # Only sample every 5th frame in order to fit in model max-token restrictions
  videos[videoID] = video[::5]

# Test videos (optional)


In [8]:
# np array with shape (frames, height, width, channels)
video = videos["720p10fps"]

fig = plt.figure()
im = plt.imshow(video[0,:,:,:])

plt.close() # this is required to not display the generated image

def init():
    im.set_data(video[0,:,:,:])

def animate(i):
    im.set_data(video[i,:,:,:])
    return im

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],
                               interval=500)
HTML(anim.to_html5_video())

# Misc. Functions

In [10]:
''' Given a conversation that is waiting for an assistant response
    and a corresponding video, return the assistant's response '''
def generate_response(conversation, video, max_new_tokens = 100, do_sample= True, top_p = 0.9):
  prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
  inputs = processor([prompt], videos=video, padding=True, return_tensors="pt").to(model.device)
  print(len(processor.tokenizer.tokenize(prompt)))
  generate_kwargs = {"max_new_tokens": max_new_tokens, "do_sample": do_sample, "top_p": top_p}
  output = model.generate(**inputs, **generate_kwargs)
  generated_text = processor.batch_decode(output, skip_special_tokens=True)
  return generated_text[0][len(prompt) - 7 + 1:]

In [11]:
'''Given a conversation where it is the user's turn to ask another question,
   the corresponding video, and the next desired question, return the assistan't
   response and the corresponding complete conversation. '''
def ask_question(conversation, video, question):
  next_question = {
      "role": "user",
          "content": [
              {"type": "text", "text": question},
          ],
  }
  conversation.append(next_question)
  response = generate_response(conversation, video)
  assistant_response = {
      "role": "assistant",
      "content": [
          {"type": "text", "text": response},
      ],
  }
  conversation.append(assistant_response)
  return conversation, response

In [12]:
import random
''' Given a conversation, number of math questions we want to ask, and video in
    the existing conversation, return the conversation with the math questions
    asked. '''
def ask_math_questions(conversation, numQuestions, video):
  for i in range(numQuestions):
    num1 = random.randint(1, 10)
    num2 = random.randint(1, 10)
    next_question = {
      "role": "user",
          "content": [
              {"type": "text", "text": "What is " + str(num1) + " + " + str(num2) +
               "? Please answer in the form \"" +  str(num1) + " + " + str(num2) + " = sum\" with no more and no less words."},
          ],
    }
    conversation.append(next_question)
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = processor([prompt], padding=True, return_tensors="pt").to(model.device)
    generate_kwargs = {"max_new_tokens": 100, "do_sample": True, "top_p": 0.90}
    output = model.generate(**inputs, **generate_kwargs)
    generated_text = processor.batch_decode(output, skip_special_tokens=True)
    response =  generated_text[0][len(prompt) + 1:]
    assistant_response = {
      "role": "assistant",
      "content": [
          {"type": "text", "text": response},
      ],
    }
    conversation.append(assistant_response)
  return conversation

In [13]:
''' As the series of questions for a certain video. Can specify the verb used
    in the Loftus question, the number of trials, and the number of math questions
    to ask as padding '''

def run_experiment(videoID, verb, numTrials = 1, numMathQuestions = 0):
  # load in the videos/questions and set out the output format
  video = videos[videoID]
  outputs = {}
  questions = [ "Give an account of the accident you have just seen in the provided video.",
              "Does the video happen at day or at night?",
              "What is the weather in the video?",
              "What color were the vehicles involved in the accident?",
              "Were any pedestrians involved in the accident?",
              "From the options, pick the single option that best estimates how fast were the cars going when they "+verb+" each other \n" +
                "A) 0 - 10 kilometers per hour\n" +
                "B) 10 - 20 kilometers per hour\n" +
                "C) 20 - 30 kilometers per hour\n" +
                "D) 30 - 40 kilometers per hour\n"]
  outputs["videoID"] = []
  outputs["verb"] = []
  outputs["trial #"] = []
  outputs["padding"] = []
  for i in range(numTrials):
    # Save info about parameters
    outputs["videoID"].append(videoID);
    outputs["verb"].append(verb)
    outputs["trial #"].append(i)
    outputs["padding"].append(numMathQuestions)
    # Generate conversation
    conversation = []
    conversation = ask_math_questions(conversation, numMathQuestions, video)
    conversation.append(
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Does a car accident occur in the provided video?"},
                {"type": "video"}, # Only 1st question has video token
                ],
        }
    )
    # Ask preliminary question about whether a car accident is in the video
    generated_response = generate_response(conversation, video)
    if "Does a car accident occur in the provided video?" in outputs:
      outputs["Does a car accident occur in the provided video?"].append(generated_response)
    else:
      outputs["Does a car accident occur in the provided video?"] = [generated_response]
    assistant_response = {
        "role": "assistant",
        "content": [
            {"type": "text", "text": generated_response},
        ],
    }
    conversation.append(assistant_response)
    # Ask remaining questions
    for question in questions:
      conversation, response = ask_question(conversation, video, question);
      if question in outputs:
        outputs[question].append(response)
      else:
        outputs[question] = [response]
  #Return outputs dictionary
  return outputs




# Experiment

In [18]:
# Arrays containing all the possible Loftus verbs and the amount of padding
verbs = ["smashed into", "collided with", "bumped into", "contacted", "hit"]
numMath = [0, 30, 60]
outputs = {}
firstTime = True;

# Iterate through all verbs and padding to run the experiment for each
for videoID in videoIDs:
  for verb in verbs:
    for num in numMath:
      o = run_experiment(videoID, verb, 1, num)
      if firstTime:
        outputs = o
        firstTime = False
      else:
        for key in o:
          outputs[key].append(o[key])

21
88
151
174
200
229
336
1226


KeyboardInterrupt: 

In [16]:
o

{'videoID': ['720p10fps'],
 'verb': ['smashed into'],
 'trial #': [0],
 'padding': [30],
 'Does a car accident occur in the provided video?': ['Yes, a car accident appears to be happening in the video. The camera pans around a busy street where a car is seen moving very fast, driving in reverse and colliding with another car. The accident results in significant damage to the vehicle on the road.'],
 'Give an account of the accident you have just seen in the provided video.': ["The video captures a car accident that occurs on a busy street. In the scene, a car drives very quickly in reverse, colliding with another car, which is seen with significant damage as a result of the impact. The camera focuses on the crash, providing a close-up view of the aftermath of the collision. It's a dangerous situation where the car in reverse is at risk of causing a more severe accident if it doesn't stop or slow down, and the scene"],
 'Does the video happen at day or at night?': ['The video happens du

In [17]:
import pandas as pd
pd.DataFrame(o)

,videoID,verb,trial #,padding,Does a car accident occur in the provided video?,Give an account of the accident you have just seen in the provided video.,Does the video happen at day or at night?,What is the weather in the video?,What color were the vehicles involved in the accident?,Were any pedestrians involved in the accident?,"From the options, pick the single option that best estimates how fast were the cars going when they smashed into each other \nA) 0 - 10 kilometers per hour\nB) 10 - 20 kilometers per hour\nC) 20 - 30 kilometers per hour\nD) 30 - 40 kilometers per hour\n"
0,720p10fps,smashed into,0,30,"Yes, a car accident appears to be happening in...",The video captures a car accident that occurs ...,The video happens during the day.,"The video depicts cloudy weather, suggesting i...",The vehicles involved in the accident are red.,"No, the video doesn't show any pedestrians bei...",B) 10 - 20 kilometers per hour
